### My solution for Pet Adoption Hackathon - Hackerearth

Importing Libraries

In [1045]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

Creating new dataframe for both train and test data.

In [1046]:
df = pd.read_csv('train.csv')
dft = pd.read_csv('test.csv')

In [1047]:
df

,pet_id,issue_date,listing_date,condition,color_type,length(m),height(cm),X1,X2,breed_category,pet_category
0,ANSL_69903,2016-07-10 00:00:00,2016-09-21 16:25:00,2.0,Brown Tabby,0.80,7.78,13,9,0.0,1
1,ANSL_66892,2013-11-21 00:00:00,2018-12-27 17:47:00,1.0,White,0.72,14.19,13,9,0.0,2
2,ANSL_69750,2014-09-28 00:00:00,2016-10-19 08:24:00,NaN,Brown,0.15,40.90,15,4,2.0,4
3,ANSL_71623,2016-12-31 00:00:00,2019-01-25 18:30:00,1.0,White,0.62,17.82,0,1,0.0,2
4,ANSL_57969,2017-09-28 00:00:00,2017-11-19 09:38:00,2.0,Black,0.50,11.06,18,4,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...
18829,ANSL_51738,2017-01-26 00:00:00,2018-03-09 15:35:00,2.0,Tricolor,0.44,27.36,0,1,0.0,2
18830,ANSL_59900,2016-06-18 00:00:00,2017-07-09 08:37:00,NaN,Brown,0.73,14.25,15,4,2.0,4
18831,ANSL_53210,2010-07-21 00:00:00,2018-08-22 14:27:00,0.0,Calico Point,0.99,28.13,13,9,1.0,1
18832,ANSL_63468,2017-05-12 00:00:00,2018-02-08 14:05:00,0.0,Tan,0.55,44.82,13,9,1.0,2


Treating the outliers that are present in the color_type column

In [1048]:
o = df['color_type'].value_counts().rename_axis('colors').reset_index(name='counts')
o.set_index(['colors'], inplace =True)
m = o[o['counts']<21].index.to_list()
def clean_color(x):
    """ If the value is a string, then remove currency symbol and delimiters
    otherwise, the value is numeric and can be converted
    """
    if x in m:
        return(x.replace(x, 'allinone'))
    return(x)

df['color_type'] = df['color_type'].apply(clean_color)

def clean_color(x):
    """ If the value is a string, then remove currency symbol and delimiters
    otherwise, the value is numeric and can be converted
    """
    if x in m:
        return(x.replace(x, 'allinone'))
    return(x)

dft['color_type'] = dft['color_type'].apply(clean_color)

Creating new features from the issue_date and listing_date by the difference between them.

In [1049]:
df['issue_date']= pd.to_datetime(df['issue_date'])
df['listing_date']= pd.to_datetime(df['listing_date'])
df['duration_days'] = abs((df['listing_date'] - df['issue_date']).dt.days)


dft['issue_date']= pd.to_datetime(dft['issue_date'])
dft['listing_date']= pd.to_datetime(dft['listing_date'])
dft['duration_days'] = (dft['listing_date'] - dft['issue_date']).dt.days


Treating the outliers of the duration_days.

In [1050]:
summary = df.describe()
Q1 = summary.iloc[4, 7]
Q3 = summary.iloc[6, 7]
mean = summary.iloc[1, 7]
IQR = Q3 - Q1
l_outliers = Q1 - (1.5 * IQR)
h_outliers = Q3 + (1.5 * IQR)
df['duration_days'] = np.where(df['duration_days'] <l_outliers, round(mean), df['duration_days'])
df['duration_days'] = np.where(df['duration_days'] >h_outliers, round(mean), df['duration_days'])
    

Filling the null values with new data

In [1051]:
df['condition'].fillna('3', inplace = True)
df.head()
dft['condition'].fillna('3', inplace = True)
dft.head()

,pet_id,issue_date,listing_date,condition,color_type,length(m),height(cm),X1,X2,duration_days
0,ANSL_75005,2005-08-17,2017-09-07 15:35:00,0,Black,0.87,42.73,0,7,4404
1,ANSL_76663,2018-11-15,2019-05-08 17:24:00,1,Orange Tabby,0.06,6.71,0,1,174
2,ANSL_58259,2012-10-11,2018-04-02 16:51:00,1,Black,0.24,41.21,0,7,1999
3,ANSL_67171,2015-02-13,2018-04-06 07:25:00,1,Black,0.29,8.46,7,1,1148
4,ANSL_72871,2017-01-18,2018-04-26 13:42:00,1,Brown,0.71,30.92,0,7,463


Creating dummies for the condition (One Hot Encoding)

In [1052]:
pet_condition = pd.get_dummies(df['condition'], prefix='condition', drop_first = True)
df = pd.concat([df, pet_condition], axis=1)
pet_condition = pd.get_dummies(dft['condition'], prefix='condition', drop_first = True)
dft = pd.concat([dft, pet_condition], axis=1)

Creating dummies for the color_type (One Hot Encoding)

In [1053]:
color = pd.get_dummies(df['color_type'], prefix='color', drop_first = True)
df = pd.concat([df, color], axis=1)
color = pd.get_dummies(dft['color_type'], prefix='color', drop_first = True)
dft = pd.concat([dft, color], axis=1)

Creating new features from issue_date column

In [1054]:
df.issue_date = df.issue_date.apply(pd.to_datetime)
df.listing_date = df.listing_date.apply(pd.to_datetime)
df['issue_year'] = df['issue_date'].dt.year 
df['issue_month'] = df['issue_date'].dt.month 
df['issue_day'] = df['issue_date'].dt.day
df['listing_month'] = df['listing_date'].dt.month 
df['listing_day'] = df['listing_date'].dt.day
df['listing_hour'] = df['listing_date'].dt.hour 
df['listing_minute'] = df['listing_date'].dt.minute
df

,pet_id,issue_date,listing_date,condition,color_type,length(m),height(cm),X1,X2,breed_category,...,color_White,color_Yellow,color_allinone,issue_year,issue_month,issue_day,listing_month,listing_day,listing_hour,listing_minute
0,ANSL_69903,2016-07-10,2016-09-21 16:25:00,2,Brown Tabby,0.80,7.78,13,9,0.0,...,0,0,0,2016,7,10,9,21,16,25
1,ANSL_66892,2013-11-21,2018-12-27 17:47:00,1,White,0.72,14.19,13,9,0.0,...,1,0,0,2013,11,21,12,27,17,47
2,ANSL_69750,2014-09-28,2016-10-19 08:24:00,3,Brown,0.15,40.90,15,4,2.0,...,0,0,0,2014,9,28,10,19,8,24
3,ANSL_71623,2016-12-31,2019-01-25 18:30:00,1,White,0.62,17.82,0,1,0.0,...,1,0,0,2016,12,31,1,25,18,30
4,ANSL_57969,2017-09-28,2017-11-19 09:38:00,2,Black,0.50,11.06,18,4,0.0,...,0,0,0,2017,9,28,11,19,9,38
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18829,ANSL_51738,2017-01-26,2018-03-09 15:35:00,2,Tricolor,0.44,27.36,0,1,0.0,...,0,0,0,2017,1,26,3,9,15,35
18830,ANSL_59900,2016-06-18,2017-07-09 08:37:00,3,Brown,0.73,14.25,15,4,2.0,...,0,0,0,2016,6,18,7,9,8,37
18831,ANSL_53210,2010-07-21,2018-08-22 14:27:00,0,allinone,0.99,28.13,13,9,1.0,...,0,0,1,2010,7,21,8,22,14,27
18832,ANSL_63468,2017-05-12,2018-02-08 14:05:00,0,Tan,0.55,44.82,13,9,1.0,...,0,0,0,2017,5,12,2,8,14,5


Creating new features from listing_date column

In [1055]:
dft.issue_date = dft.issue_date.apply(pd.to_datetime)
dft.listing_date = dft.listing_date.apply(pd.to_datetime)
dft['issue_year'] = dft['issue_date'].dt.year 
dft['issue_month'] = dft['issue_date'].dt.month 
dft['issue_day'] = dft['issue_date'].dt.day
dft['listing_month'] = dft['listing_date'].dt.month 
dft['listing_day'] = dft['listing_date'].dt.day
dft['listing_hour'] = dft['listing_date'].dt.hour 
dft['listing_minute'] = dft['listing_date'].dt.minute
dft

,pet_id,issue_date,listing_date,condition,color_type,length(m),height(cm),X1,X2,duration_days,...,color_White,color_Yellow,color_allinone,issue_year,issue_month,issue_day,listing_month,listing_day,listing_hour,listing_minute
0,ANSL_75005,2005-08-17,2017-09-07 15:35:00,0,Black,0.87,42.73,0,7,4404,...,0,0,0,2005,8,17,9,7,15,35
1,ANSL_76663,2018-11-15,2019-05-08 17:24:00,1,Orange Tabby,0.06,6.71,0,1,174,...,0,0,0,2018,11,15,5,8,17,24
2,ANSL_58259,2012-10-11,2018-04-02 16:51:00,1,Black,0.24,41.21,0,7,1999,...,0,0,0,2012,10,11,4,2,16,51
3,ANSL_67171,2015-02-13,2018-04-06 07:25:00,1,Black,0.29,8.46,7,1,1148,...,0,0,0,2015,2,13,4,6,7,25
4,ANSL_72871,2017-01-18,2018-04-26 13:42:00,1,Brown,0.71,30.92,0,7,463,...,0,0,0,2017,1,18,4,26,13,42
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8067,ANSL_66809,2016-02-10,2017-03-10 14:56:00,2,Brown,0.82,36.08,13,9,394,...,0,0,0,2016,2,10,3,10,14,56
8068,ANSL_59041,2015-12-07,2018-02-12 00:00:00,0,Tan,0.49,27.54,13,9,798,...,0,0,0,2015,12,7,2,12,0,0
8069,ANSL_60034,2015-12-08,2017-01-04 17:19:00,0,Black,0.98,37.19,0,7,393,...,0,0,0,2015,12,8,1,4,17,19
8070,ANSL_58066,2016-06-28,2017-07-20 18:19:00,3,Black,0.79,23.83,0,2,387,...,0,0,0,2016,6,28,7,20,18,19


In [1056]:
X = df

X_test = dft

In [1057]:
m = [cname for cname in X.columns if X[cname].dtype == "object" or 
                        X[cname].dtype == "datetime64[ns]"]
X.drop(m, axis = 1, inplace = True)
X_test.drop(m, axis = 1, inplace = True)

In [1058]:
k = ['breed_category', 'pet_category']
y = df.breed_category
X.drop(k, axis = 1, inplace =True)

Spliting train and test data

In [1059]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2,
                                                                random_state=31)


Using Lightgbm model for training the data

In [1060]:
import lightgbm as lgb
my_model_3 = lgb.LGBMClassifier(boosting_type='dart', learning_rate=0.01, n_estimators=132, random_state = 31).fit(X_train,y_train)
predictions_3 = my_model_3.predict(X_valid)
predictions_3 = np.around(predictions_3, decimals = 0)
mae_3 = f1_score(predictions_3, y_valid, average='weighted')
print("Mean Absolute Error:" , mae_3)

Mean Absolute Error: 0.9130049421174751


Testing the test data

In [1061]:
preds_test = my_model_3.predict(X_test)
preds_test = np.around(preds_test, decimals = 0)

In [1062]:
dft1 = pd.read_csv('test.csv')

Now doing the same feature engineering with some changes for predicting the pet_category as follows

In [1063]:
df = pd.read_csv('train.csv')
dft = pd.read_csv('test.csv')

In [1064]:
df['issue_date']= pd.to_datetime(df['issue_date'])
df['listing_date']= pd.to_datetime(df['listing_date'])
df['duration_days'] = abs((df['listing_date'] - df['issue_date']).dt.days)


dft['issue_date']= pd.to_datetime(dft['issue_date'])
dft['listing_date']= pd.to_datetime(dft['listing_date'])
dft['duration_days'] = abs((dft['listing_date'] - dft['issue_date']).dt.days)



In [1065]:
df['condition'].fillna('3', inplace = True)
df.head()
dft['condition'].fillna('3', inplace = True)
dft.head()

,pet_id,issue_date,listing_date,condition,color_type,length(m),height(cm),X1,X2,duration_days
0,ANSL_75005,2005-08-17,2017-09-07 15:35:00,0,Black,0.87,42.73,0,7,4404
1,ANSL_76663,2018-11-15,2019-05-08 17:24:00,1,Orange Tabby,0.06,6.71,0,1,174
2,ANSL_58259,2012-10-11,2018-04-02 16:51:00,1,Black,0.24,41.21,0,7,1999
3,ANSL_67171,2015-02-13,2018-04-06 07:25:00,1,Black,0.29,8.46,7,1,1148
4,ANSL_72871,2017-01-18,2018-04-26 13:42:00,1,Brown,0.71,30.92,0,7,463


In [1066]:
pet_condition = pd.get_dummies(df['condition'], prefix='condition', drop_first = True)
df = pd.concat([df, pet_condition], axis=1)
pet_condition = pd.get_dummies(dft['condition'], prefix='condition', drop_first = True)
dft = pd.concat([dft, pet_condition], axis=1)

In [1067]:
color = pd.get_dummies(df['color_type'], prefix='color', drop_first = True)
df = pd.concat([df, color], axis=1)
color = pd.get_dummies(dft['color_type'], prefix='color', drop_first = True)
dft = pd.concat([dft, color], axis=1)

In [1068]:
df.issue_date = df.issue_date.apply(pd.to_datetime)
df.listing_date = df.listing_date.apply(pd.to_datetime)
df['issue_year'] = df['issue_date'].dt.year 
df['issue_month'] = df['issue_date'].dt.month 
df['issue_day'] = df['issue_date'].dt.day
df['listing_year'] = df['listing_date'].dt.year 
df['listing_month'] = df['listing_date'].dt.month 
df['listing_day'] = df['listing_date'].dt.day
df['listing_hour'] = df['listing_date'].dt.hour 
df['listing_minute'] = df['listing_date'].dt.minute
df

,pet_id,issue_date,listing_date,condition,color_type,length(m),height(cm),X1,X2,breed_category,...,color_Yellow,color_Yellow Brindle,issue_year,issue_month,issue_day,listing_year,listing_month,listing_day,listing_hour,listing_minute
0,ANSL_69903,2016-07-10,2016-09-21 16:25:00,2,Brown Tabby,0.80,7.78,13,9,0.0,...,0,0,2016,7,10,2016,9,21,16,25
1,ANSL_66892,2013-11-21,2018-12-27 17:47:00,1,White,0.72,14.19,13,9,0.0,...,0,0,2013,11,21,2018,12,27,17,47
2,ANSL_69750,2014-09-28,2016-10-19 08:24:00,3,Brown,0.15,40.90,15,4,2.0,...,0,0,2014,9,28,2016,10,19,8,24
3,ANSL_71623,2016-12-31,2019-01-25 18:30:00,1,White,0.62,17.82,0,1,0.0,...,0,0,2016,12,31,2019,1,25,18,30
4,ANSL_57969,2017-09-28,2017-11-19 09:38:00,2,Black,0.50,11.06,18,4,0.0,...,0,0,2017,9,28,2017,11,19,9,38
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18829,ANSL_51738,2017-01-26,2018-03-09 15:35:00,2,Tricolor,0.44,27.36,0,1,0.0,...,0,0,2017,1,26,2018,3,9,15,35
18830,ANSL_59900,2016-06-18,2017-07-09 08:37:00,3,Brown,0.73,14.25,15,4,2.0,...,0,0,2016,6,18,2017,7,9,8,37
18831,ANSL_53210,2010-07-21,2018-08-22 14:27:00,0,Calico Point,0.99,28.13,13,9,1.0,...,0,0,2010,7,21,2018,8,22,14,27
18832,ANSL_63468,2017-05-12,2018-02-08 14:05:00,0,Tan,0.55,44.82,13,9,1.0,...,0,0,2017,5,12,2018,2,8,14,5


In [1069]:
dft.issue_date = dft.issue_date.apply(pd.to_datetime)
dft.listing_date = dft.listing_date.apply(pd.to_datetime)
dft['issue_year'] = dft['issue_date'].dt.year 
dft['issue_month'] = dft['issue_date'].dt.month 
dft['issue_day'] = dft['issue_date'].dt.day
dft['listing_year'] = dft['listing_date'].dt.year 
dft['listing_month'] = dft['listing_date'].dt.month 
dft['listing_day'] = dft['listing_date'].dt.day
dft['listing_hour'] = dft['listing_date'].dt.hour 
dft['listing_minute'] = dft['listing_date'].dt.minute
dft

,pet_id,issue_date,listing_date,condition,color_type,length(m),height(cm),X1,X2,duration_days,...,color_Yellow,color_Yellow Brindle,issue_year,issue_month,issue_day,listing_year,listing_month,listing_day,listing_hour,listing_minute
0,ANSL_75005,2005-08-17,2017-09-07 15:35:00,0,Black,0.87,42.73,0,7,4404,...,0,0,2005,8,17,2017,9,7,15,35
1,ANSL_76663,2018-11-15,2019-05-08 17:24:00,1,Orange Tabby,0.06,6.71,0,1,174,...,0,0,2018,11,15,2019,5,8,17,24
2,ANSL_58259,2012-10-11,2018-04-02 16:51:00,1,Black,0.24,41.21,0,7,1999,...,0,0,2012,10,11,2018,4,2,16,51
3,ANSL_67171,2015-02-13,2018-04-06 07:25:00,1,Black,0.29,8.46,7,1,1148,...,0,0,2015,2,13,2018,4,6,7,25
4,ANSL_72871,2017-01-18,2018-04-26 13:42:00,1,Brown,0.71,30.92,0,7,463,...,0,0,2017,1,18,2018,4,26,13,42
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8067,ANSL_66809,2016-02-10,2017-03-10 14:56:00,2,Brown,0.82,36.08,13,9,394,...,0,0,2016,2,10,2017,3,10,14,56
8068,ANSL_59041,2015-12-07,2018-02-12 00:00:00,0,Tan,0.49,27.54,13,9,798,...,0,0,2015,12,7,2018,2,12,0,0
8069,ANSL_60034,2015-12-08,2017-01-04 17:19:00,0,Black,0.98,37.19,0,7,393,...,0,0,2015,12,8,2017,1,4,17,19
8070,ANSL_58066,2016-06-28,2017-07-20 18:19:00,3,Black,0.79,23.83,0,2,387,...,0,0,2016,6,28,2017,7,20,18,19


In [1070]:
X = df

X_test = dft

In [1071]:
m = [cname for cname in X.columns if X[cname].dtype == "object" or 
                        X[cname].dtype == "datetime64[ns]"]
X.drop(m, axis = 1, inplace = True)
X_test.drop(m, axis = 1, inplace = True)
k = ['breed_category', 'pet_category', 'color_Brown Tiger', 'color_Black Tiger']
y = df.pet_category
X.drop(k, axis = 1, inplace =True)

In [1072]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2,
                                                                random_state=31)


In [1073]:
import lightgbm as lgb
my_model_3 = lgb.LGBMClassifier(boosting_type='dart', learning_rate=0.095, n_estimators=201, random_state = 31).fit(X_train,y_train)
predictions_3 = my_model_3.predict(X_valid)
predictions_3 = np.around(predictions_3, decimals = 0)
mae_3 = f1_score(predictions_3, y_valid, average='weighted')
print("Mean Absolute Error:" , mae_3)


Mean Absolute Error: 0.9112827899530328


In [1074]:
preds_test1 = my_model_3.predict(X_test)
preds_test1 = np.around(preds_test1, decimals = 0)
output = pd.DataFrame({'pet_id': dft1['pet_id'], 'breed_category': preds_test, 'pet_category': preds_test1})
output.to_csv('my_submission1.csv', index=False)